# Advanced options

In the previous lesson we learned about the basic and intermediate options of Tensor Fox. For most of applications this is enough, but sometimes one needs to change more parameters, add constraints, and so on. In this lesson we also cover the options regarding higher order tensors. Warning: this lesson has a more mathematical flavour.

Options already covered:

    display
    maxiter  
    tol     
    tol_step
    tol_improv
    tol_grad
    tol_mlsvd
    svd_n_components
    trunc_dims
    initialization
    refine    
    init_damp
    symm    
    tol_jump
    
Options to be covered:

    method
    inner_method 
    cg_maxiter 
    cg_factor
    cg_tol 
    constraints 
    trials 
    bi_method
    bi_method_maxiter 
    bi_method_tol 
    epochs 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import TensorFox as tfx
from IPython.display import Image

class options:    # start creating an empty class
    pass

options = tfx.make_options(options)    # update the class

In [2]:
# Create the tensor.
m = 2
T = np.zeros((m, m, m))
s = 0

for k in range(m):
    for i in range(m):
        for j in range(m):
            T[i,j,k] = s
            s += 1

# Inner algorithm options

The method we are using to solve the problem of tensor approximation is called *damped Gauss-Newton* (dGN), and at each step of this method the program needs to solve a equation of the form

$$(J^T J + \mu D) x = J^Tb$$
as already mentioned. To solve this equation we have to rely on other method, which can be an iterative method like the [conjugate gradient](https://en.wikipedia.org/wiki/Conjugate_gradient_method) (default) or a direct method using matrix factorization. The conjugate gradient methods has its own parameters, and the user may have to tune them sometimes. With this in mind, Tensor Fox offers the parameters $\verb|inner| \_ \verb|method|, \ \verb|cg| \_ \verb|maxiter|, \ \verb|cg| \_ \verb|tol|$ and $\verb|cg| \_ \verb|factor|$. They are explained below.

The inner methods are: $\verb|cg|, \ \verb|cg| \_ \verb|static|$, $\verb|direct|$ and $\verb|als|$ (alternating least squares, but this one doesn't take in account the regularization). We also mention that it is possible to pass the parameter  $\verb|inner| \_ \verb|method|$ as a list of strings containing the names of the method available. Then the program uses the prescribed sequence of methods, one at each iterarion. We noticed that this hybrid way of work can bring good results sometimes. 

The difference between static and non-static versions are the way the program deals with the maximum number of iterations. The static algorithm have a certain maximum number of iterations $\verb|cg| \_ \verb|maxiter|$ which is fixed during all the program. The non-static versions uses the parameter $\verb|cg| \_ \verb|factor|$ to control the number of iterations in a different way. If the program is at the $k$-th iteration of the dGN, then the maximum number of iterations permitted for the cg method is
 
$$1 + int\left( \verb|cg|\_\verb|factor| \cdot \verb|randint|\left( 1 + k^{0.4}, 2 + k^{0.9} \right) \right).$$

This strange interval of random integers were obtained after a lot of tests, a lot! This seems to be a robust choice, but since we can't be right all the time, the parameter $\verb|cg| \_ \verb|factor|$ comes to the rescue. If the number of maximum iterations are increasing too much, just set this parameter to a low value such as $0.1$ or $0.5$. Finally, the parameter $\verb|cg| \_ \verb|tol|$, as the name suggests, is the tolerance parameter for the cg method. The cg iterations stops when the (absolute) residual is less than $\verb|cg| \_ \verb|tol|$. Below there is an example showing how to setup a method and its parameters.

In [3]:
# Let's use cg_static as the inner algorithm, with 3 iterations max and tolerance of 1e-7.
options.inner_method = 'cg_static'
options.cg_maxiter = 3
options.cg_tol = 1e-7
options.display = 2

R = 3
factors, output = tfx.cpd(T, R, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    No compression detected
    Working with dimensions (2, 2, 2)
-----------------------------------------------------------------------------------------------
Type of initialization: random
-----------------------------------------------------------------------------------------------
Computing CPD
    Iteration | Rel error |  Step size  | Improvement | norm(grad) | Predicted error | # Inner iterations
        1     | 1.91e-01  |  1.35e+00   |  1.91e-01   |  8.24e+00  |    3.04e-02     |        3        
        2     | 1.40e-01  |  3.14e-02   |  5.08e-02   |  6.06e+00  |    3.75e-05     |        3        
        3     | 1.07e-01  |  4.54e-02   |  3.39e-02   |  2.28e+00  |    9.05e-05     |        3        
        4     | 8.46e-02  |  4.76e-02   |  2.20e-02   |  6.80e-01  |

       104    | 1.91e-13  |  1.53e-13   |  4.58e-14   |  2.62e-12  |    3.42e-27     |        3        
       105    | 1.56e-13  |  1.39e-13   |  3.56e-14   |  1.88e-12  |    3.38e-27     |        3        
       106    | 1.22e-13  |  1.69e-13   |  3.39e-14   |  2.45e-12  |    5.02e-27     |        3        
       107    | 7.01e-14  |  1.12e-13   |  5.16e-14   |  2.92e-12  |    1.04e-27     |        3        
       108    | 6.31e-14  |  2.20e-14   |  6.98e-15   |  7.03e-13  |    2.46e-28     |        3        
       109    | 5.24e-14  |  3.76e-14   |  1.08e-14   |  6.75e-13  |    3.23e-28     |        3        
       110    | 4.72e-14  |  1.65e-14   |  5.20e-15   |  5.10e-13  |    1.34e-28     |        3        
       111    | 3.99e-14  |  2.60e-14   |  7.27e-15   |  4.88e-13  |    1.92e-28     |        3        
       112    | 3.59e-14  |  1.24e-14   |  4.06e-15   |  3.92e-13  |    8.21e-29     |        3        
       113    | 2.93e-14  |  2.35e-14   |  6.52e-15   |  3.83e-1

# Constraints

The parameter $\verb|factors| \_ \verb|norm|$ is used to fix the norm of the factor matrices of the CPD. Suppose $T$ is a third tensor and $(X^{(k)}, Y^{(k)}, Z^{(k)})$ the approximated CPD at iteration $k$. If one set $\verb|factors| \_ \verb|norm| = 2$, for example, then $\| X^{(k)} \| = \| Y^{(k)} \| = \| Z^{(k)} \| = 2$ for all $k$.

# Higher order tensors and the Tensor Train format

Tensor Fox has distinct approaches when it comes to computing the CPD of third order tensors and higher order tensors. By default the program relies on the *Damped Gauss-Newton* (dGN) method. However you can set the program to use the *Tensor Train format* (TT format), also called *Tensor Train decomposition*. Without going in too much details, we use a specific configuration of the TT format which can be obtained by computing several third order CPD's. More precisely, if $T$ is a tensor of order $L$, then we can compute a CPD for it by computing $L-2$ third order CPD's. Once we have the TT format of $T$, the CPD can also be computed. The figure below illustrate the representation of a tensor train associated to a tensor of order $L$.

![tensortrain](tensor-train.png)

Each square represent the coordinates of a tensor and each circle is the coordinate with is shared between two consecutive tensors. These tensors are usually denoted by $\mathcal{G}^{(\ell)}$. For example, the second tensor is $\mathcal{G}^{(2)}$, which has coordinates $\mathcal{G}^{(\ell)}_{j_1 i_2 j_2}$, and the next tensor is $\mathcal{G}^{(3)}$, which has coordinates $\mathcal{G}^{(\ell)}_{j_2 i_3 j_3}$. The first and last tensor are acutally matrices (which mean $j_0 = j_L = 1$), and the other $L-2$ tensor are third order tensors. They are related to $T$ by the following formula:
$$T_{i_1 i_2 \ldots i_L} = \sum_{j_0, j_1, \ldots, j_L} \mathcal{G}^{(1)}_{j_0 i_1 j_1} \cdot \mathcal{G}^{(2)}_{j_1 i_2 j_2} \cdot \ldots \cdot \mathcal{G}^{(L)}_{j_{L-1} i_L j_L}.$$

By computing a CPD for $\mathcal{G}^{(2)}, \ldots, \mathcal{G}^{(L-1)}$ we can obtain a CPD for $T$.

## Trials

In the case $T$ has order higher than $3$, the parameter $\verb|trials|$ defines how much times we compute each one of these third order CPD's. The idea is to compute several times and keep the best result (smaller error). This may be helpful because all $L-2$ CPD's needs to be of good quality in order to get a good CPD for $T$. If just one of the third order CPD's has bad precision, than everything falls apart. Currently the default is $\verb|trials|= 3$, but this may change depending on the problem. This parameter doesn't makes difference if $T$ is a third order tensor. 

## Display

As we've said, the options $\verb|trials|$ says about the repetition of third order CPD computations. If $\verb|display|$ is set to $1, 2, 3$ or $4$, then all the information of each one of these CPD's are printed on the screen. This means we wil have $(L-2) \cdot \verb|trials|$ CPD's informations printed on the screen when $T$ has order $L$ . Sometimes this amount of information is just too much. We can make everything more succint in these situations just by setting $\verb|display| =-1$. Consider the following fourth order tensor.

In [4]:
# Initialize dimensions of the tensor.
k = 2
dims = (k+1, k+1, k+1, k+1)
L = len(dims)

# Create four random factors matrices so that
# A = (orig_factors[0], orig_factors[1], orig_factors[2], orig_factors[3])*I.
orig_factors = []
for l in range(L):
    M = np.random.randn(dims[l], k)
    Q, R = np.linalg.qr(M)
    orig_factors.append(Q)
    
# From the factor matrices generate the respective tensor in coordinates.
A = tfx.cpd2tens(orig_factors)

print('A = ')
tfx.showtens(A) # now this is the same as print(A)

A = 
[[[[-0.00764225  0.01205816 -0.17680627]
   [-0.16744559 -0.11133222  0.18654112]
   [-0.12195923 -0.05699456 -0.12465404]]

  [[ 0.11587787  0.07708424 -0.12951077]
   [-0.08607661 -0.07550637  0.29349431]
   [ 0.10572895  0.05821381  0.01287093]]

  [[-0.16968183 -0.12735558  0.34620885]
   [ 0.27228081  0.2088145  -0.60369104]
   [-0.05769106 -0.04034286  0.0857318 ]]]


 [[[-0.09483383 -0.05260432 -0.0073353 ]
   [-0.03540679 -0.00932205 -0.11430334]
   [-0.15683356 -0.08014238 -0.08623113]]

  [[ 0.02429232  0.00633343  0.0790969 ]
   [ 0.08119444  0.05084477 -0.05649959]
   [ 0.08807856  0.04267411  0.07366732]]

  [[ 0.0431817   0.03480818 -0.11403312]
   [-0.0935093  -0.06941095  0.18243344]
   [-0.0014035   0.00283097 -0.0391363 ]]]


 [[[-0.2628439  -0.1349533  -0.13760548]
   [-0.20767392 -0.09943114 -0.18652892]
   [-0.5074391  -0.25575758 -0.31733555]]

  [[ 0.14314548  0.06851535  0.12879286]
   [ 0.1634003   0.08794477  0.04176088]
   [ 0.30976182  0.15473047  0.208

In [5]:
# Compute the CPD of A with succint display for higher order tensors.
class options:    
    pass

options = tfx.make_options(options)    
options.display = -1
options.method = 'ttcpd'
    
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compression detected
    Compressing from (3, 3, 3, 3) to (2, 2, 2, 2)

Total of 2 third order CPDs to be computed:
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 1 error = 4.515082331623521e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 2 error = 6.832608714974581e-16

Final results
    Number of steps = 203
    Relative error = 7.48881350176193e-16
    Accuracy =  100.0 %


In [6]:
# The options display = -2 is showed below. 
options.display = -2
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compression detected
    Compressing from (3, 3, 3, 3) to (2, 2, 2, 2)
    Compression relative error = 1.151339e-15

SVD Tensor train error =  7.586455687746145e-16

Total of 2 third order CPDs to be computed:
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 1 error = 9.510588827099324e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 2 error = 8.622116668970984e-16

CPD Tensor train error =  0.8328104967585118

Final results
    Number of steps = 151
    Relative error = 1.886636496638417e-15
    Accuracy =  100.0 %


## MLSVD tolerance with high order tensors

Now let's see what happens when we set $\verb|tol| \_ \verb|mlsvd| = [\verb|1e-6|, \verb|-1|]$ for the tensor $A$. This choice means the program will perform the high order compression using $10^{-6}$ as tolerance, and will not compress the intermediate third order tensors. 

In [7]:
options.tol_mlsvd = [1e-6, -1]
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compression detected
    Compressing from (3, 3, 3, 3) to (2, 2, 2, 2)
    Compression relative error = 1.061020e-15

SVD Tensor train error =  7.011395207591572e-16

Total of 2 third order CPDs to be computed:
CPD 1 error = 1.197838807422973e-16
CPD 2 error = 3.3663863383104695e-15

CPD Tensor train error =  4.4332246850836

Final results
    Number of steps = 151
    Relative error = 3.35449413266312e-15
    Accuracy =  100.0 %


## Inner algorithm options

Just as usual third order tensors has the options $\verb|method|, \verb|tol|, \verb|maxiter|$ for its inner computations, the third order tensors of a tensor train also can receive these parameters. However there is a difference here: when computing the CPD's of each $\mathcal{G}^{(\ell)}$, the program starts computing the CPD of $\mathcal{G}^{(2)}$, and one factor is of the CPD is used to compute the CPD of $\mathcal{G}^{(3)}$. Then one factor of this CPD is used to compute the CPD of $\mathcal{G}^{(4)}$ and so on. In short, each CPD depends on the previous computed CPD. The matrices $\mathcal{G}^{(1)}$ and $\mathcal{G}^{(L)}$ are easily computed after we have the other CPD's.

The first CPD can be computed as any CPD, but the other always depends on some previous computed factor, which is always used to fix one factor of the next CPD. This means each CPD, except the first, is actually only computing two factors, so there is a difference in how the program computes the first CPD and the remaining ones. Therefore, the parameters $\verb|method|, \ \verb|method| \_ \verb|tol|, \ \verb|method| \_ \verb|maxiter|$ are used for the first CPD and the parameters $\verb|bi| \_ \verb|method|, \ \verb|bi| \_ \verb|method| \_ \verb|tol|, \ \verb|bi| \_ \verb|method| \_ \verb|maxiter|$ are used for all the remaining CPD's. The figure below illustrate these observations.

![tensortrainmethods](tensor-train-methods.png)

## Epochs

As we can note, there is a flow of information in the tensor train format, the CPD's are computed from left to right, and the next CPD always depend on some information about the previous CPD. Once we compute the CPD of $\mathcal{G}^{(L-1)}$ it is possible to "go back", that is, use the information of the CPD of $\mathcal{G}^{(L-1)}$ to compute a new CPD for $\mathcal{G}^{(L-2)}$, we just have to reverse the way information is propagated. Doing this we may be able to refine all CPD's. These cycles can repeated several times, with the information being propagated forward and backward again and again. Each cycle is called an *epoch*, and the number of epochs can be passed to the program through the parameter $\verb|epochs|$. Below we redefine the tensor $A$ to have a higher order and try to refine the CPD by using more epochs than just $1$.

In [8]:
# Initialize the sixth-order tensor and compute its CPD with default options.
k = 2
dims = (k+1, k+1, k+1, k+1, k+1, k+1)
L = len(dims)

orig_factors = []
for l in range(L):
    M = np.random.randn(dims[l], k)
    Q, R = np.linalg.qr(M)
    orig_factors.append(Q)
    
A = tfx.cpd2tens(orig_factors)

class options:    
    pass

options = tfx.make_options(options)    
options.display = -1
options.method = 'ttcpd'
    
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compressing unfolding mode 5
    Compressing unfolding mode 6
    Compression detected
    Compressing from (3, 3, 3, 3, 3, 3) to (2, 2, 2, 2, 2, 2)

Total of 4 third order CPDs to be computed:
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 1 error = 5.62419791799874e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 2 error = 5.996919198000854e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 3 error = 8.925586375671795e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 4 error = 7.432070425192828e-16

Final results
    

In [9]:
# Now we use 5 epochs on the same tensor.
options.epochs = 5
options.display = -1
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compressing unfolding mode 5
    Compressing unfolding mode 6
    Compression detected
    Compressing from (3, 3, 3, 3, 3, 3) to (2, 2, 2, 2, 2, 2)

Total of 4 third order CPDs to be computed:
Epoch  1
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 1 error = 2.4192517009910737e-15
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 2 error = 5.493655113344711e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 3 error = 8.601989796748051e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 4 error = 9.879581447690234e-16

Epoch  

# Method

The last parameter to be seen is $\verb|method|$. By default Tensor Fox uses $\verb|method| = \verb|'dGN'|$, which means the program will use the damped Gauss-Newton method. For higher order tensors, $\verb|method| = \verb|'ttcpd'|$ can be a good choice when the rank is smaller than all dimensions. This method is recommended for dense tensors since it avoids the curse of dimensionality. We note that it is also possible to set $\verb|method| = \verb|'ttcpd'|$ for third order tensors. Finally, another possibility is $\verb|method| = \verb|'als'|$ (Alternating Least Squares), a classic and well known method to compute CPDs.